# Pandas tricks & pitfalls

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('mini_movie_data.csv')

df.head()

In [ ]:
df.describe()

# isin

# The `rename` function

In [ ]:
# rename the 'movie' column to 'title'.
# you can rename multiple columns by adding more key:value pairs to the dictionary
df.rename(columns={'movie':'title'}, inplace=True)
df.head()

# unique

In [ ]:
# how many unique studio names are there?
print len(df.studio.unique())

In [ ]:
print df.studio.unique()

In [ ]:
# unique values will not be sorted, you have to do it yourself
print sorted(df.studio.unique())

# Groupby objects

In [ ]:
actors = df.groupby('actor')
# this is a groupby object. do not be scared. it is your friend.
actors

In [ ]:
actors.mean().head()

In [ ]:
# calling size() on a groupby object will return the number of rows each group contains.
# here, how many roles each actor has
actors.size().head(10)

In [ ]:
# agg() can take a list of functions. 
# It makes a new column and applies them to each group in a groupby
actors['domestic_gross','worldwide_gross'].agg(['mean','count','std','min','max']).head(10)

In [ ]:
# ASIDE: which female actors appear most often in the dataset?
top_actresses = df[df.male==0].groupby('actor').size().sort_values(ascending=False).head()
top_actresses

In [ ]:
# often we want to select all rows where a column contains any value in a list
# eg, select all rows where df.actor is in our list of actors
actor_list = ['Susan Sarandon','Julia Roberts']
# This won't work:
# df[df.actor in actor_list]

In [ ]:
# instead, use pandas.DataFrame.isin:
df[df.actor.isin(actor_list)].head()

# pd.to_numeric()

Converts a series, array, or dataframe to a numeric datatype.

In [ ]:
# example DataFrame of numbers-as-strings
num_example = pd.DataFrame(data=zip(list('2049204795'),list('6185700963')), columns=['a','b'])
num_example

In [ ]:
# if you add columns a and b, they're just concatenated together because they're strings!
num_example.a + num_example.b

In [ ]:
# apply pd.to_numeric across the whole dataframe to convert everything to numeric values
num_numeric = num_example.apply(pd.to_numeric)
num_numeric

In [ ]:
# now adding the columns actually gives you the sum
num_numeric.a + num_numeric.b

In [ ]:
# this example illustrates 2 things:
# 1) grouping based on a conditional statement (is an even number)
# 2) iterating through groups in a groupby
for name, group in num_numeric.groupby(num_numeric.a%2==0):
    print name, '\n', group
    print '* * *'

# Working with Timestamps

In [ ]:
# recall what the actor info dataframe looks like
df.head()

In [ ]:
# what is the data type (dtype) of the bday column?
df.bday.dtype

In [ ]:
# we can also print an element of the column to look at it
df.bday[0]

In [ ]:
# we can also check the type of the first element
type(df.bday[0])

## pd.to_datetime

In [ ]:
# convert the columns of date-time strings to pandas Timestamp objects (similar to to_numeric)
# we don't use .apply here because we only want to change these 2 specified columns
for datetime_col in ['bday','release_date']:
    df[datetime_col] = pd.to_datetime(df[datetime_col])    

In [ ]:
df.bday.dtype

In [ ]:
type(df.bday[0])

## Instant conversion to day/month/year with 
### `pd.Series.dt.<day/month/year/second/etc>`

In [ ]:
print 'years', df.bday.dt.year.unique()

In [ ]:
# this doesn't work.
# df[df.bday > 1995]

In [ ]:
# instead you could compare to a Timestamp or other datetime object
df[df.bday > pd.to_datetime('1-1-1995')].head()

In [ ]:
# or, use the .dt syntax:
df[df.bday.dt.year > 1995].head()

In [ ]:
# Pitfall!
# when you want to select using multiple conditions, watch out for this pandas pitfall
# (this doesn't work:)
# df[2000 > df.bday.dt.year > 1995].head()

In [ ]:
# Pitfall!
# Instead, use the bitwise and (&) operator. However...
# (this doesn't work either):
# df[2000 > df.bday.dt.year & df.bday.dt.year > 1995].head()

### Since the '`&`' operator has really high precedence in order of operations, be sure to enclose each condition in *parentheses*.

Eg: `2000 > df.bday.dt.year & df.bday.dt.year > 1995` is evaluated the same as 

`2000 > (df.bday.dt.year & df.bday.dt.year) > 1995`


In [ ]:
# select birthdays between 1995 and 2000, non-inclusive
df[(2000 > df.bday.dt.year) & (df.bday.dt.year > 1995)].head()

In [ ]:
# example of .dt.month
# Note: you rarely need to add columns like this!! You can use .dt directly for a groupby or for a selection
df2 = df.copy()
df2['release_month'] = df2.release_date.dt.month
df2.head()

In [ ]:
monthly_mean = df.groupby(df.release_date.dt.month).mean()
monthly_mean

In [ ]:
monthly_mean[['domestic_gross','worldwide_gross']].plot.bar(title='Mean monthly gross')

In [ ]:
# you don't need to make a new column for a one-off.
(monthly_mean.domestic_gross / monthly_mean.worldwide_gross).plot.bar(
    title='Mean Domestic/Worldwide Gross Ratio by month')

## But that's gross, we don't want month numbers on the x axis, but the month names instead

`calendar` library to the rescue

In [ ]:
import calendar

# we have the option of full name of month, or abbreviated name
print calendar.month_name[1:4]
print calendar.month_abbr[1:4]

In [ ]:
# map over the the index of using calendar's month names
monthly_mean.index = monthly_mean.index.map(lambda x: calendar.month_abbr[x])
monthly_mean

In [ ]:
# now we have month abbreviations as x labels when we plot
(monthly_mean.domestic_gross / monthly_mean.worldwide_gross).plot.bar(
    title='Mean Domestic/Worldwide Gross Ratio by month')

# The `resample` method

A convenient way to bin timeseries data

**Warning:** resample only works with a Timestamp-indexed dataframe. You can always set your index to your datetime column of interest `df.set_index('datetime_column')` to make this work

In [ ]:
# let's look at movies of a given actor, by year
actor_df = df[df.actor=='Samuel L. Jackson'].drop('male', axis=1)
actor_df.sort_values('release_date').head()

In [ ]:
# visualize what the data looks like now: it's irregular by year
actor_df.plot('release_date','production_budget')

In [ ]:
# take the mean of all the numerical columns
actor_df.set_index('release_date').resample('AS', how='mean').head()

## note that by default, missing bins get replaced with a NaN row. This is can be useful if you want to set a default value to the missing bins.

In [ ]:
# same as above, but fill all NaNs with 0
actor_df.set_index('release_date').resample('AS', how='mean').fillna(0).head()

In [ ]:
# if we want 5-year bins instead, we can plug in a 5 to the resample "rule": '5AS'
actor_df.set_index('release_date').resample('5AS', how='mean')

## resample resolutions available [(via SO answer)](http://stackoverflow.com/a/17001474):

    B       business day frequency
    C       custom business day frequency (experimental)
    D       calendar day frequency
    W       weekly frequency
    M       month end frequency
    BM      business month end frequency
    CBM     custom business month end frequency
    MS      month start frequency
    BMS     business month start frequency
    CBMS    custom business month start frequency
    Q       quarter end frequency
    BQ      business quarter endfrequency
    QS      quarter start frequency
    BQS     business quarter start frequency
    A       year end frequency
    BA      business year end frequency
    AS      year start frequency
    BAS     business year start frequency
    BH      business hour frequency
    H       hourly frequency
    T       minutely frequency
    S       secondly frequency
    L       milliseonds
    U       microseconds
    N       nanoseconds

In [ ]:
# let's say we want the mean, and also the count.
# we can pass a list of methods to the `how`
yr_bins = actor_df.set_index('release_date').resample('5AS', how=['mean','count','sem'])
yr_bins.head()

In [ ]:
# or you can get very fancy and pass a dict of dicts
# the first key references the DataFrame's original column name
# the second key defines the name of a new column.
yr_bins = actor_df.set_index('release_date').resample('5AS', how={
        'production_budget':{'avg':'mean', 'ct':'count', 'stdEm':'sem'},
        'domestic_gross':{'low':'min', 'high':'max'},
        'worldwide_gross':{'total':'sum'}})
yr_bins

# Special note: try not to use method names as column names. It will make indexing more annoying.
## For example, a column named 'mean' will cause a collision when you call `df.mean`
## The `mean` method will have precedence.

You'd only be able to access the column like: `df['mean']`

In [ ]:
# PS: 'sem' is standard error of the mean
# pd.Series.sem?

# Multiindexing

In [ ]:
yr_bins.production_budget

In [ ]:
# chaining the dot column name syntax is fine
yr_bins.production_budget.avg

In [ ]:
# you can also index both levels of the column index by name, as strings
yr_bins['production_budget','avg']

## Flattening a multi-level column index

### Use a list comprehension to rewrite the column names

In [ ]:
print yr_bins.columns.values

In [ ]:
yr_bins_flat = yr_bins.copy()
# use an underscore as a delimiter. But it's up to you.
yr_bins_flat.columns = ['_'.join(col) for col in yr_bins.columns.values]

yr_bins_flat

# `pd.cut()`: bins numeric values -> categorical values

In [ ]:
# make some fake data
no_movies = 10
ratings_df = pd.DataFrame.from_dict({
    'rating_no':pd.np.random.rand(no_movies), 
    'movie':df.title.sample(no_movies)})
# fake gross based on fake rating
ratings_df['gross'] = pd.np.round(ratings_df.rating_no*100000000, decimals=2)

# save this unmodified version for later
ratings_df_orig = ratings_df.copy()

ratings_df

In [ ]:
# cut numerical ratings into N bins

# here's what the labels default to when you don't define your own labels
ratings_df['rating_category_ugly'] = pd.cut(ratings_df.rating_no, bins=4)

# you can substitute whatever labels you want
ratings_df['rating_category'] = pd.cut(ratings_df.rating_no, bins=4, labels=['bad','mediocre','good','excellent'])

ratings_df

In [ ]:
# `pd.cut` gives us an excellent way to groupby based on bins.
# Eg, we can use the new categorical ratings to find the mean gross for each rating bin
print 'mean gross for each rating bin'
ratings_df.groupby('rating_category').mean()

In [ ]:
# Even if we didn't care about assigning labels like 'bad', 'mediocre', etc to the rating numbers,
# pd.cut is still very useful if we want to groupby on binned numerical data

# We can do this as a one-liner, using the copy of the original ratings_df before we added those extra columns.
# Let's do 5 bins to switch it up.
ratings_df_orig.groupby(pd.cut(ratings_df_orig.rating_no, bins=5)).mean()

### Just like with `resample`, empty bins have *NaN* values.

In [ ]:
ratings_df_orig.groupby(pd.cut(ratings_df_orig.rating_no, bins=5), as_index=False).mean()

# Bonus: Taking advantage of seaborn's groupby support

In [ ]:
n_top = 15
# we only want one row per movie, we don't care about actors
by_movie_df = df.groupby('title').first()
by_movie_df.head()

In [ ]:
# select only the top N studios, by total production budget of all movies
top_studio_names = by_movie_df.groupby('studio').sum().sort_values(
    'production_budget', ascending=False).index[:n_top]

top_studio_df = by_movie_df[by_movie_df.studio.isin(top_studio_names)]

print top_studio_names
top_studio_df.head()

In [ ]:
import seaborn as sns

# make the size of the figure bigger (width,height)
plt.figure(figsize=(14,8))

# we pass the studio column to sns.violinplot
sns.violinplot(top_studio_df.production_budget, groupby=top_studio_df.studio)
plt.title('Production budget distributions for the top 10 studios');